In [4]:
# import libraries
import pandas as pd
from sqlalchemy import create_engine
import nltk
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tokenize import word_tokenize
import re
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
from sklearn.multioutput import MultiOutputClassifier
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV
from Custom_Transformers import MessageTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [5]:
# load data from database
engine = create_engine('sqlite:///Vasilis_db.db')
df = pd.read_sql_table('Emergency_Messages', engine)
df = df.drop(columns = ['id','original'], axis=1)

In [6]:
# drop rows with NaN messages
df = df.dropna(subset=['message'])

In [7]:
df = df.dropna()

In [8]:
df.loc[df.related==2, 'related'] = 1

In [9]:
X = df[['message','genre']]
Y = df.drop(columns=['message','genre'], axis=1)

In [10]:
def tokenize(text):
    """ Function that tokenizes and lemmatizes text

    :param text:     input text to be processed(str)
    :return:         cleaned_tokens(str)
    """

    stop_words = stopwords.words("english")
    lemmatizer = WordNetLemmatizer()

    # Text normalization
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())
    #tokenization
    words = word_tokenize (text)
    # lemmatization
    cleaned_tokens = [lemmatizer.lemmatize(word).strip() for word in words if word not in stop_words]

    return cleaned_tokens 

In [12]:
text_transformer = Pipeline([
    ('messagetransformer', MessageTransformer(tokenize)),
])
genre_transformer = Pipeline([
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])
preprocessor = ColumnTransformer([
    ('text', text_transformer, [0]),
    ('genre', genre_transformer, [1])
])
pipeline_2 = Pipeline([
    ('preprocess', preprocessor),
    ('clf', RandomForestClassifier(random_state=33))
])

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, random_state=33)
pipeline_2.fit(X_train, y_train)

Pipeline(steps=[('preprocess',
                 ColumnTransformer(transformers=[('text',
                                                  Pipeline(steps=[('messagetransformer',
                                                                   MessageTransformer(tokenize=<function tokenize at 0x7fb7ec418d90>))]),
                                                  [0]),
                                                 ('genre',
                                                  Pipeline(steps=[('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  [1])])),
                ('clf', RandomForestClassifier(random_state=33))])

In [14]:
preds = pipeline_2.predict(X_test)

ValueError: X has 14906 features, but DecisionTreeClassifier is expecting 27355 features as input.